In [2]:
import openai
import json
from sklearn.metrics import classification_report
import pandas as pd
from tqdm import tqdm
from credentials import OPENAPI_KEY
openai.api_key = OPENAPI_KEY

In [ ]:
train_ds_path = "DS_PATH/train.jsonl"
openai.File.create(file=open(train_ds_path, "rb"), purpose="fine-tune")

In [ ]:
openai.FineTuningJob.create(training_file="FILE_ID", model="gpt-3.5-turbo")

In [ ]:
openai.FineTuningJob.retrieve("FINETUNE_ID")

In [40]:
test_ds_path = "DS_PATH/test.jsonl"
with open(test_ds_path, "r") as f:
    test_dataset = [json.loads(line) for line in f]

In [41]:
test_dataset[0]["messages"]

[{'role': 'system',
  'content': 'vrl-gpt3.5-turbo is a chatbot that can give the sentiment of Turkish texts.'},
 {'role': 'user',
  'content': 'What is the sentiment of this Turkish text "Ulan Wifi\'ye bağlıyım ben. Ona bağlıyken Turkcell internet paketin bitti diye nasıl mesaj atabilir bana ya? Onu da mı ödeyelim"?'},
 {'role': 'assistant', 'content': 'negative'}]

In [ ]:
preds = []
refs = []
for example in tqdm(test_dataset):
    completion = openai.ChatCompletion.create(
        model="MODEL_ID",
        messages=example["messages"][:2]
        )
    model_output = completion.choices[0].message
    ground_truth = example["messages"][-1]["content"]
    preds.append(model_output["content"])
    refs.append(ground_truth)

In [ ]:
output_df = pd.DataFrame({"preds": preds, "refs": refs})
output_df.to_csv("OUTPUT_PATH/output.csv", index=False)

In [4]:
temp_df  = pd.read_csv("OUTPUT_PATH/output.csv")
temp_df.head()

,preds,refs
0,neutral,negative
1,neutral,negative
2,neutral,negative
3,neutral,negative
4,negative,negative


In [5]:
temp_df["preds"].value_counts()

preds
neutral     2754
negative    1018
positive     958
Positive       1
Neutral        1
relief         1
Name: count, dtype: int64

In [6]:
refs, preds = temp_df["refs"], temp_df["preds"]
output_labels = ["negative", "neutral", "positive"]
for index in range(len(preds)):
    if preds[index] not in output_labels:
        preds[index] = "neutral" 

In [7]:
print(classification_report(refs, preds, digits=4))

              precision    recall  f1-score   support

    negative     0.4126    0.6061    0.4909       693
     neutral     0.7044    0.7013    0.7029      2769
    positive     0.4207    0.3171    0.3616      1271

    accuracy                         0.5842      4733
   macro avg     0.5125    0.5415    0.5185      4733
weighted avg     0.5855    0.5842    0.5802      4733

